In [1]:
import requests
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
from urllib.parse import quote

import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.alert import Alert

from datetime import datetime, timedelta

# 크롤링

In [6]:
glob(r'C:\workspace\City_Rock\바꾼후\*.xlsx')

['C:\\workspace\\City_Rock\\바꾼후\\임시1.xlsx',
 'C:\\workspace\\City_Rock\\바꾼후\\임시2.xlsx']

In [11]:
file = glob(r'C:\workspace\City_Rock\바꾼후\*.xlsx')[0]
base = pd.read_excel(file, engine='openpyxl')
base

,■ 일별 박스오피스,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
0,- 조회일: 2023-05-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,- 출처: 영화진흥위원회 통합전산망 (http://www.kobis.or.kr),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,▶ 검색조건 [ 조회기간: 2023-05-10~2023-05-17 영화구분: 전...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,●2023년 05월 17일(수),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
869,94,2023년 4월 수요단편극장,NaN,8000,0,8000,1,71000,1,1,...,1,1,한국,한국,NaN,NaN,NaN,기타,NaN,NaN
870,94,"안녕, 티라노: 영원히, 함께",2019-08-14 00:00:00,4000,0,0,0,1591206229,1,0,...,1,1,한국,한국,(주)미디어캐슬,(주)넥스트엔터테인먼트월드(NEW),전체관람가,애니메이션,시즈노 코분,"시영준,정혜원,이세은,이현,김지혜,박영진,김준현,사성웅,시영준,정형석,곽윤상,이동훈..."
871,94,건설현장의 여직원을 마사지로 유혹,NaN,6000,0,6000,1,6000,1,1,...,1,1,일본,일본,NaN,(주)도키엔터테인먼트,청소년관람불가,성인물(에로),니시다 류지,NaN
872,106,라스트 버스,2023-02-09 00:00:00,0,0,0,1,21799091,0,0,...,1,1,영국,영국,NaN,주식회사 블루필름웍스,12세이상관람가,드라마,질리스 맥키넌,"티모시 스폴,필리스 로건"


In [225]:
# 셀레니움으로 url 주소 크롬 열기
driver = webdriver.Chrome()
url = 'https://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do'
driver.get(url)
time.sleep(1)

# 초기 날짜 지정
start_day = datetime.strptime("2006-01-01", '%Y-%m-%d')

while True:

    start_day_str = start_day.strftime('%Y-%m-%d')

    # 조회기간 시작일 비우기
    driver.find_element(By.NAME,'sSearchFrom').clear()
    time.sleep(0.5)
    Alert(driver).accept()
    time.sleep(0.5)
    
    # 조회기간 마지막일 비우기
    driver.find_element(By.NAME,'sSearchTo').clear()
    time.sleep(0.5)
    Alert(driver).accept()
    time.sleep(0.5)
    
    # 조회기간 시작일 값 입력
    driver.find_element(By.NAME,'sSearchFrom').send_keys(start_day_str)
    time.sleep(0.5)
    driver.find_element(By.XPATH,'/html/body').click()
    time.sleep(0.5)

    # 조회 버튼 클릭
    driver.find_element(By.CSS_SELECTOR,'div.wrap_btn .btn_blue').click()
    time.sleep(2)

    # 엑셀 파일 다운로드
    driver.find_element(By.CSS_SELECTOR,'div.tar a:nth-child(1)').click()
    time.sleep(2)

    # 자바스크립트 확인
    Alert(driver).accept()
    time.sleep(3)
    
    # 시작일 변경
    start_day = start_day + timedelta(days=7)
    
    # 최종일 지정
    if start_day == datetime.strptime("2007-12-09", '%Y-%m-%d'):
        break

# 파일 합치기

In [2]:
import pandas as pd
from glob import glob
import warnings
warnings.filterwarnings('ignore')

In [297]:
col_list = ['순위', '영화명', '개봉일', '매출액', '매출액점유율', 
            '매출액증감(전일대비)', '매출액증감율(전일대비) ','누적매출액 ', 
            '관객수 ', '관객수증감(전일대비) ', '관객수증감율(전일대비) ', 
            '누적관객수 ','스크린수 ', '상영횟수 ', '대표국적 ', '국적 ', 
            '제작사 ', '배급사 ', '등급 ', '장르 ','감독 ', '배우 ','기준일']

In [298]:
df1 = pd.DataFrame(columns = col_list)

for file in glob(r'C:\Users\tjoeun\Desktop\2006년\*.xlsx'):
    base = pd.read_excel(file, engine='openpyxl')
    
    try: 

        ii = base[base["Unnamed: 4"].isnull()].index[range(7,len(base[base["Unnamed: 4"].isnull()].index),3)]
        ii_lis = [7]

        for i in range(len(ii)):
            ii_lis.append(int(ii[i])+3)

        df2 = pd.DataFrame(columns = col_list)

        for jj in range(len(ii_lis)):
            if jj != len(ii_lis)-1:
                df = base.iloc[ii_lis[jj]:ii_lis[jj+1]-4]
                df["기준일"] = datetime.strptime(base.iloc[ii_lis[jj]-3][0].split("●")[1].split("(")[0],'%Y년 %m월 %d일')
                df.columns = col_list
                df2 = pd.concat([df,df2],axis=0)

            else:
                df = base.iloc[ii_lis[jj]:]
                df["기준일"] = datetime.strptime(base.iloc[ii_lis[jj]-3][0].split("●")[1].split("(")[0],'%Y년 %m월 %d일')
                df.columns = col_list
                df2 = pd.concat([df,df2],axis=0)
        df1 = pd.concat([df1,df2],axis=0)
    
    except:
        pass

In [200]:
start_day

datetime.datetime(2008, 1, 6, 0, 0)

In [201]:
start_day + timedelta(days=7)

datetime.datetime(2008, 1, 13, 0, 0)

In [286]:
datetime.strptime("2012-01-01", '%Y-%m-%d') - datetime.strptime("2006-01-01", '%Y-%m-%d')

datetime.timedelta(days=2191)

In [223]:
707/7

101.0

In [224]:
212+101

313

# 파일 확인

In [299]:
df1

,순위,영화명,개봉일,매출액,매출액점유율,매출액증감(전일대비),매출액증감율(전일대비),누적매출액,관객수,관객수증감(전일대비),...,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,기준일
241,1,왕의 남자,2005-12-29 00:00:00,1978748500,0.369,-112303500,-0.054,15691240400,310347,-19859,...,1685,한국,한국,"(주)씨네월드,(주)이글픽쳐스",(주)시네마서비스,15세이상관람가,드라마,이준익,"감우성,이준기,유해진,정진영,박영수,강성연,이건우,이동수,이동헌,유효정,윤준식,박성...",2006-01-08
242,2,싸움의 기술,2006-01-05 00:00:00,1011112000,0.189,-62605500,-0.058,3194795000,154904,-11033,...,1477,한국,한국,코리아 엔터테인먼트,CJ ENM,15세관람가,코미디,신한솔,"백윤식,재희,김응수,허종수,송승현,김무유,최여진,박종하,이동진,변승현,한정훈,장성현...",2006-01-08
243,3,"나니아 연대기-사자,마녀 그리고 옷장",2005-12-29 00:00:00,837768000,0.156,-65769000,-0.073,9293011500,135889,-11422,...,1176,미국,미국,월트 디즈니 픽쳐스,브에나비스타인터내셔널코리아,전체관람가,"가족,어드벤처,판타지",앤드류 아담슨,"조지 헨리,스캔더 킨즈",2006-01-08
244,4,킹콩,2005-12-14 00:00:00,644062500,0.12,-31513000,-0.047,17868618900,103477,-5690,...,783,미국,미국,유니버셜 픽쳐스,UIP코리아,15세관람가,"드라마,어드벤처",피터 잭슨,"잭 블랙,애드리언 브로디,나오미 왓츠",2006-01-08
245,5,작업의 정석,2005-12-21 00:00:00,339977000,0.063,-24893500,-0.068,10635155600,52048,-4096,...,750,한국,한국,영화사청어람(주),(주)쇼박스,15세관람가,코미디,오기환,"손예진,송일국,김애경,박용우,안선영,박준규,신주연,신준하,서민희,고재욱,윤영준,정민...",2006-01-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,29,인 굿 컴퍼니,2005-08-26 00:00:00,25000,0,25000,1,1212370000,5,5,...,1,미국,미국,유니버셜 픽쳐스,스폰지,15세관람가,"코미디,드라마",폴 웨이츠,"토퍼 그레이스,데니스 퀘이드",2006-01-07
36,29,파랑주의보,2005-12-22 00:00:00,26000,0,-115500,-0.816,1453692000,5,-16,...,2,한국,한국,(주)아이필름홀딩스,아이러브시네마,12세관람가,드라마,전윤수,"송혜교,차태현,김신영,김윤석,김지영,김해숙,한명구,최우혁,송창의,조미리,홍성준,박정...",2006-01-07
37,29,도쿄타워,2005-11-23 00:00:00,25000,0,25000,1,325979000,5,5,...,1,일본,일본,"토쿄 브로드캐스팅 시스팀,도에이",스폰지,18세관람가,드라마,미나모토 타카시,"한다 켄토,테라지마 시노부,오카다 준이치,쿠로키 히토미",2006-01-07
38,32,연애사진,2004-04-09 00:00:00,10000,0,-15000,-0.6,39437500,2,-3,...,1,일본,일본,쇼지쿠 컴퍼니,(주)스폰지이엔티,15세관람가,드라마,츠츠미 유키히코,"히로스에 료코,마츠다 류헤이",2006-01-07


In [281]:
df1.to_csv("2006-2011.csv", index=False)

In [300]:
imsi = df1.groupby("기준일")["영화명"].count().to_frame().reset_index()
imsi['기준연도'] = imsi['기준일'].apply(lambda x:x.year)
imsi['기준월'] = imsi['기준일'].apply(lambda x:x.month)
imsi['기준연도'].unique()

array([2006, 2007, 2008, 2009, 2010, 2011], dtype=int64)

In [301]:
 imsi['기준월'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int64)

In [302]:
df1[df1['순위'] == '합계']

,순위,영화명,개봉일,매출액,매출액점유율,매출액증감(전일대비),매출액증감율(전일대비),누적매출액,관객수,관객수증감(전일대비),...,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,기준일
271,합계,NaN,NaN,5355440700,NaN,NaN,NaN,107330181200,840154,NaN,...,7540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-01-08
236,합계,NaN,NaN,2252607300,NaN,NaN,NaN,111969970000,360671,NaN,...,7252,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-01-09
199,합계,NaN,NaN,2263988100,NaN,NaN,NaN,113164303100,363700,NaN,...,7212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-01-10
164,합계,NaN,NaN,2224625900,NaN,NaN,NaN,115411498000,360227,NaN,...,7201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-01-11
131,합계,NaN,NaN,2289619900,NaN,NaN,NaN,188214966909,374490,NaN,...,7168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-01-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,합계,NaN,NaN,2688197400,NaN,NaN,NaN,95633454100,433889,NaN,...,7061,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-01-03
146,합계,NaN,NaN,2434855200,NaN,NaN,NaN,99692286800,395023,NaN,...,6988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-01-04
108,합계,NaN,NaN,2592579400,NaN,NaN,NaN,94457437200,421108,NaN,...,7314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-01-05
73,합계,NaN,NaN,2961390800,NaN,NaN,NaN,97819153000,473952,NaN,...,7691,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-01-06


In [303]:
 df1[df1['순위'] == 1]

,순위,영화명,개봉일,매출액,매출액점유율,매출액증감(전일대비),매출액증감율(전일대비),누적매출액,관객수,관객수증감(전일대비),...,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,기준일
241,1,왕의 남자,2005-12-29 00:00:00,1978748500,0.369,-112303500,-0.054,15691240400,310347,-19859,...,1685,한국,한국,"(주)씨네월드,(주)이글픽쳐스",(주)시네마서비스,15세이상관람가,드라마,이준익,"감우성,이준기,유해진,정진영,박영수,강성연,이건우,이동수,이동헌,유효정,윤준식,박성...",2006-01-08
204,1,왕의 남자,2005-12-29 00:00:00,1064643000,0.473,-914105500,-0.462,16755883400,170152,-140195,...,1578,한국,한국,"(주)씨네월드,(주)이글픽쳐스",(주)시네마서비스,15세이상관람가,드라마,이준익,"감우성,이준기,유해진,정진영,박영수,강성연,이건우,이동수,이동헌,유효정,윤준식,박성...",2006-01-09
169,1,왕의 남자,2005-12-29 00:00:00,1073217000,0.474,8574000,0.008,17829100400,171707,1555,...,1574,한국,한국,"(주)씨네월드,(주)이글픽쳐스",(주)시네마서비스,15세이상관람가,드라마,이준익,"감우성,이준기,유해진,정진영,박영수,강성연,이건우,이동수,이동헌,유효정,윤준식,박성...",2006-01-10
136,1,왕의 남자,2005-12-29 00:00:00,1067077500,0.48,-6139500,-0.006,18896177900,170866,-841,...,1581,한국,한국,"(주)씨네월드,(주)이글픽쳐스",(주)시네마서비스,15세이상관람가,드라마,이준익,"감우성,이준기,유해진,정진영,박영수,강성연,이건우,이동수,이동헌,유효정,윤준식,박성...",2006-01-11
89,1,왕의 남자,2005-12-29 00:00:00,1014911000,0.443,-52166500,-0.049,19911088900,164120,-6746,...,1625,한국,한국,"(주)씨네월드,(주)이글픽쳐스",(주)시네마서비스,15세이상관람가,드라마,이준익,"감우성,이준기,유해진,정진영,박영수,강성연,이건우,이동수,이동헌,유효정,윤준식,박성...",2006-01-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,1,왕의 남자,2005-12-29 00:00:00,1039432600,0.387,-75951200,-0.068,8392221700,168268,-11653,...,1381,한국,한국,"(주)씨네월드,(주)이글픽쳐스",(주)시네마서비스,15세이상관람가,드라마,이준익,"감우성,이준기,유해진,정진영,박영수,강성연,이건우,이동수,이동헌,유효정,윤준식,박성...",2006-01-03
113,1,왕의 남자,2005-12-29 00:00:00,980763200,0.403,-58669400,-0.056,9372984900,159539,-8729,...,1394,한국,한국,"(주)씨네월드,(주)이글픽쳐스",(주)시네마서비스,15세이상관람가,드라마,이준익,"감우성,이준기,유해진,정진영,박영수,강성연,이건우,이동수,이동헌,유효정,윤준식,박성...",2006-01-04
78,1,왕의 남자,2005-12-29 00:00:00,1028675000,0.397,47911800,0.049,10401659900,166954,7415,...,1479,한국,한국,"(주)씨네월드,(주)이글픽쳐스",(주)시네마서비스,15세이상관람가,드라마,이준익,"감우성,이준기,유해진,정진영,박영수,강성연,이건우,이동수,이동헌,유효정,윤준식,박성...",2006-01-05
44,1,왕의 남자,2005-12-29 00:00:00,1219780000,0.412,191105000,0.186,11621439900,195957,29003,...,1605,한국,한국,"(주)씨네월드,(주)이글픽쳐스",(주)시네마서비스,15세이상관람가,드라마,이준익,"감우성,이준기,유해진,정진영,박영수,강성연,이건우,이동수,이동헌,유효정,윤준식,박성...",2006-01-06


In [304]:
df3 = pd.read_csv("2003-2005.csv")
df3

,순위,영화명,개봉일,매출액,매출액점유율,매출액증감(전일대비),매출액증감율(전일대비),누적매출액,관객수,관객수증감(전일대비),...,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,기준일
0,1,오페라의 유령,2004-12-08 00:00:00,785374500,0.358,-169168500.0,-0.177,2809708000,120110,-31250.0,...,967,미국,미국,워너 브러더스 픽쳐스,"(주)팝엔터테인먼트,(주)드림팩트엔터테인먼트,CJ ENM","12세이상관람가,12세관람가","뮤지컬,드라마",조엘 슈마허,"제라드 버틀러,에미 로섬,패트릭 윌슨,미란다 리차드슨",2004-12-12
1,2,브리짓 존스의 일기2 : 열정과 애정,2004-12-08 00:00:00,561298500,0.256,-99235000.0,-0.150,2022444000,83828,-16591.0,...,855,영국,영국,"워킹 타이틀 필름,유니버셜 픽쳐스",UIP코리아,15세관람가,"멜로/로맨스,드라마",비번 키드론,"르네 젤위거,휴 그랜트,린 콜린스",2004-12-12
2,3,나비효과,2004-11-12 00:00:00,212469500,0.097,-18695000.0,-0.081,5874853000,31758,-2941.0,...,550,미국,미국,필름엔진,(주)쇼박스,18세관람가,드라마,"에릭 브레스,J. 마키에 그러버","에이미 스마트,애쉬튼 커처,에릭 스톨츠,윌리엄 리 스콧,엘든 헨슨",2004-12-12
3,4,노트북,2004-11-26 00:00:00,132193500,0.060,-19705500.0,-0.130,2512605000,19764,-3481.0,...,444,미국,미국,뉴라인 시네마,"(주)퍼스트런,TCO(주)더콘텐츠온,글뫼 주식회사,CJ ENM","15세관람가,15세이상관람가",멜로/로맨스,닉 카사베츠,"레이첼 맥아덤즈,라이언 고슬링",2004-12-12
4,5,여선생 VS 여제자,2004-11-17 00:00:00,122699500,0.056,-9280500.0,-0.070,5112304506,19415,-2647.0,...,441,한국,한국,(주)좋은영화,CJ ENM,15세관람가,코미디,장규성,"김응수,이지영,이지훈,이세영,염정아,정형민,김혜경,황경애,최란,오은석,최윤정,이정헌...",2004-12-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21963,27,천국의 아이들 2 - 시험보는 날,2005-11-17 00:00:00,97000,0.000,-93000.0,-0.489,101039500,15,-20.0,...,5,이란,이란,NaN,(주)영화사한결,전체관람가,드라마,골람 레자 라메자니,"메흐다드 하사니,가잘리 파사파",2005-12-31
21964,28,나는 할렐루야 아줌마였다,1982-10-21 00:00:00,30000,0.000,18000.0,1.500,10714000,5,3.0,...,1,한국,한국,화풍흥업,씨네마오병이어(주),"연소자관람가,전체관람가",드라마,김수형,"송도영,최지원,신영일",2005-12-31
21965,합계,NaN,NaN,6855154000,NaN,NaN,NaN,86817039100,1087280,NaN,...,8108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005-12-31
21966,1,동갑내기 과외하기,2003-02-07 00:00:00,633500,1.000,633500.0,1.000,633500,102,102.0,...,1,한국,한국,코리아 엔터테인먼트,CJ ENM,12세관람가,코미디,김경형,"김하늘,권상우,공유,김지우,백일섭,김혜옥,김자옥,오승근,이성진,손영민,송병철,신지현...",2003-01-12
